# Задание №5
### по практикуму на ЭВМ (2015-2016)
#### Выполнила студентка 317 группы
#### Шолохова Татьяна
##### 24.02.2016

В задании требовалось реализовать обучение разреженного автокодировщика и показать, как он обнаруживает, что границы объектов и цветовые переходы - одно из лучших представлений для естественных изображений.

In [2]:
import numpy as np
import PIL
import math
import scipy.optimize 
import display_layer as dl
import sample_patches as sp
import autoencoder as aec
import gradient as g
from importlib import reload
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import sklearn.ensemble
import sklearn.svm

## Подготовка

### **1. Генерация выборки патчей на основе неразмеченных исходных изображений**

In [3]:
reload(sp)
def make_patches(num=10000, size=8):
    patches = np.zeros((num, size * size * 3))
    N = num
    k = 5
    i = 0
    for fname in 'X1', 'X2', 'X3', 'X4', 'X5':
        v = (N + k - 1) // k
        f = open('./data2.7/' + fname + '.pk', 'rb')
        images = pickle.load(f)
        patches[i:i + v] = sp.sample_patches(images, v, size)
        del images
        f.close()
        i += v
        N -= v
        k -= 1
    return patches

**Исходные изображения**

<img src="images.png", width=200, align=left>

In [31]:
reload(dl)
reload(sp)
data = make_patches()
dl.display_layer(data[:64], 'patches.png')

**Сгенерированные патчи**

<img src="patches.png", width=200, align=left>

### **2. Формулы для подсчета градиентов**

Функция потерь является суммой трёх слагаемых: квадратичное отклонение исходных изображений $X$ от выходных $Y$; регуляризация, ограничивающая веса коэффициентов; регуляризация разреженности среднего слоя. 
$$L = Q + R_1 + R_2$$
$$Q = \frac{1}{2N}||X-Y||_F^2$$
$$R_1 = \frac{\lambda}{2}\sum\limits_{i=0}^{l-1}||W^{(i)}||_F^2$$
$$R_2 = \beta\sum\limits_{i = 1}^{s}\left(\rho \log\frac{\rho}{\hat\rho_i} + (1 - \rho) \log\frac{1 - \rho}{1 - \hat\rho_i}\right)$$ , где $N$ &ndash; количество изображений, $l$ &ndash; количество слоёв нейросети, $||\cdot||_F$ &ndash; норма Фробениуса, $\lambda, \rho, \beta$ &ndash; параметры регуляризаций, $W^{(i)}$ &ndash; матрица весов между $i$ и $i+1$ слоями, $s$ &ndash; количество нейронов на среднем слое, $\hat\rho_j$ &ndash; среднее значение активации $j$-го нейрона на среднем слое. 

Если бы функция потерь состояла только из квадратичного отклонения исходных изображений $X$ от выходных $Y$, то градиент этой функции по параметрам нейросети можно вычислить, используя метод обратного распространения ошибки(соответствующие формулы приведены в условии задания).

Выясним как изменится градиент при добавлении только первого регуляризатора. 
$$\frac{\partial L}{\partial W^{(i)}} = \frac{\partial Q}{\partial W^{(i)}} + \frac{\partial R_1}{\partial W^{(i)}}$$
Первое слагаемое мы уже умеем вычислять, второе слагаемое можно выписать и вычислить аналитически:
$$\frac{\partial R_1}{\partial W^{(i)}} = \lambda W^{(i)}$$
Для корректного вычисления градиентов в этом случае, необходимо выполнить алгоритм обратного распространения ошибки по "старым" формулам, после чего добавить к градиентам по всем $W^{(i)}$ вычисленную добавку $\lambda W^{(i)}$.

Теперь добавим второй регуляризатор. Обозначим $S$ матрицу активаций по всем изображениям на среднем слое автокодировщика(матрица $N \times D$, $N$ &ndash; количество изображений, $D$ &ndash; размерность среднего слоя).
$$\frac{\partial L}{\partial A} = \frac{\partial Q}{\partial A} + \frac{\partial R_2}{\partial A}$$
Первое слагаемое мы уже умеем вычислять, второе слагаемое:
$$\frac{\partial R_2}{\partial A_{ij}} = \sum \limits_{k = 1}^{s} \frac{\partial R_2}{\partial \hat\rho_k} \frac{\partial \hat\rho_k}{\partial A_{ij}}$$
$$\frac{\partial R_2}{\partial \hat\rho_k} = \beta\left(-\frac{\hat\rho_k}{\rho} + \frac{1 - \hat\rho_k}{1 - \rho}\right)$$


$$
\frac{\partial \hat\rho_k}{\partial A_{ij}} = 
\left\{
\begin{aligned}
    0,& \qquad k \neq j \\
    \frac{1}{N},& \qquad k = j \\
\end{aligned}
\right.
$$

$$
    \frac{\partial R_2}{\partial A_{ij}} = \frac{\beta}{N} \left(-\frac{\hat\rho_j}{\rho} + \frac{1 - \hat\rho_j}{1 - \rho} \right)
$$

Тем самым для корректного учета градиента по $R_2$ в методе обратного распространения ошибки необходимо прибавить выписанную добавку при вычислении $\frac{\partial L}{\partial A}$

## Обучение разреженного автокодировщика

Проверка корректности функции приближенного вычисления градиента на основе разностной апроксимации. Объявленная простая функция:
$$J(\vec{x}) = \sin{(x_0 x_1 x_2)} + x_0 x_2 $$, легко убедиться, что ее градиент $$\frac{\partial{L}}{\partial{\vec{x}}} = (x_1 x_2\cos{(x_0 x_1 x_2)} + x_2~~ ,~~ x_2 x_0 \cos{(x_0 x_1 x_2)}~~ ,~~ x_0 x_1 \cos{(x_0 x_1 x_2)} + x_0)^T$$

In [4]:
reload(g)
g.check_gradient()

Ok


Погрешность не более $10^{-4}$

### **1. Вычисление функции потерь разреженного однослойного автокодировщика и соответствующего градиента с сигмоидной функцией активации и сравнение полученного градиента с приближенно вычисленным **

In [46]:
reload(aec)
hidden_size = np.array([75])
visible_size = 192
lambda_ = 1e-4
sparsity_param = 1e-2
beta = 1e-1
theta = aec.initialize(hidden_size, visible_size)
grad = aec.autoencoder_loss(theta, visible_size, hidden_size, lambda_, \
                            sparsity_param, beta, data[:10])[1]
reload(g)
J = lambda theta: aec.autoencoder_loss(theta, visible_size, hidden_size, \
                                       lambda_, sparsity_param, beta, data[:10])[0]
True_grad = g.compute_gradient(J, theta)
print(np.max(np.abs(True_grad - grad)))

2.40606782761e-09


### **2. Минимизация функции потерь разреженного однослойного автокодировщика**

In [43]:
reload(aec)
hidden_size = np.array([75])
visible_size = 192
lambda_ = 5e-4
sparsity_param = 1e-2
beta = 3
theta = aec.initialize(hidden_size, visible_size)
J = lambda theta: aec.autoencoder_loss(theta, visible_size, hidden_size, lambda_, \
                                       sparsity_param, beta, data[:10000])
optimal = scipy.optimize.minimize(J, theta, method='L-BFGS-B', jac=True, \
                                  options={'disp' : True, 'maxiter' : 2000})

## Визуализация

In [44]:
W = optimal['x']
w = W[:192*75].reshape(192, 75)
w = w.T
reload(dl)
dl.display_layer(w[:64], 'layer4.png')

**Полученные фильтры** 

|$\lambda = 10^{-4}, \rho = 10^{-2}, \beta = 3$|$\lambda = 10^{-4}, \rho = 10^{-2}, \beta = 0.1$|
| :--------------------------:|:---------------------------:|
|<img src="layer.png", width=250, align=left>  |<img src="layer2.png", width=250, align=left> |
|$\lambda = 10^{-4}, \rho = 0.1, \beta = 3$    |$\lambda = 5*10^{-4}, \rho = 10^{-2}, \beta = 3$   |
|<img src="layer3.png", width=250, align=left> |<img src="layer4.png", width=250, align=left> |
Видно, что при уменьшениии $\beta$ фильтры серееют. При увеличении $\rho$ портятся фильтры границ. При увеличении $\lambda$ нейроны почти никогда не активируются.

## Обучение классификатора на данных сокращённой размерности

**Обучение параметров однослойного автокодировщика**

In [ ]:
reload(aec)
hidden_size = np.array([75])
visible_size = 192
lambda_ = 1e-4
sparsity_param = 1e-2
beta = 3
theta = aec.initialize(hidden_size, visible_size)
J = lambda theta: aec.autoencoder_loss(theta, visible_size, hidden_size, lambda_, \
                                       sparsity_param, beta, data[:10000])
optimal = scipy.optimize.minimize(J, theta, method='L-BFGS-B', jac=True, \
                                  options={'disp' : True, 'maxiter' : 2000})
theta = optimal['x']
np.savez('theta.npz', theta=theta)

**Выделение признаков автокодировщиком**

In [5]:
f = open('./data/train.pk', 'rb')
train = pickle.load(f)
f.close()
f = open('./data/test.pk', 'rb')
test = pickle.load(f)
f.close()

Признаки для изображения из автокодировщика генерируются следующим образом: из изображения с равномерным шагом генерируются патчи фиксированного размера, после чего каждый патч пропускается до среднего слоя обученной нейросети(получая новые признаки), все признаки конкатенируются и образуют итоговые признаки для изображения. 

In [8]:
def autoencoder_features(theta, hidden_size, visible_size, data, step=8, patch_size=8):
    features = []
    pic_cnt = data.shape[0]
    pic_size = int(np.sqrt(data.shape[1] // 3))
    data = data.reshape((pic_cnt, pic_size, pic_size, 3))
    layer_number = (2 + hidden_size.size) // 2
    for i in range(0, pic_size-patch_size+1, step):
        for j in range(0, pic_size-patch_size+1, step):
            data_new = data[:, i:i+patch_size, j:j+patch_size, :].reshape(pic_cnt, patch_size*patch_size*3)
            features.append(aec.autoencoder_transform(theta, visible_size, hidden_size, \
                                                      layer_number, data_new))
    return np.hstack(features)

**Обучение рандомизированных деревьев и алгоритма опорных векторов. В качестве признаков используются значения интенсивностей цветных каналов изображений**

In [78]:
X = train['X']
y = train['y'].ravel()
trees = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=3)
svm = sklearn.svm.LinearSVC()
trees.fit(X, y)
svm.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [79]:
X_test = test['X']
y_test = test['y'].ravel()
print(np.mean(trees.predict(X_test) == y_test))
print(np.mean(svm.predict(X_test) == y_test))

0.43675
0.28375


**Обучение рандомизированных деревьев и алгоритма опорных векторов. Признаки получены с помощью однослойного автокодировщика. Шаг =  8**

In [6]:
del data
f = open('./data/train.pk', 'rb')
train = pickle.load(f)
f.close()

In [9]:
X2 = autoencoder_features(theta, hidden_size, visible_size, train['X'], step=8, patch_size=8)
y2 = train['y'].ravel()
trees2 = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=3)
svm2 = sklearn.svm.LinearSVC()
trees2.fit(X2, y2)
svm2.fit(X2, y2)

C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [10]:
del X2
del y2
del train
f = open('./data/test.pk', 'rb')
test = pickle.load(f)
f.close()

In [12]:
X_test = test['X']
y_test = test['y'].ravel()
sum_tree = 0
sum_svm = 0
for i in range(0, X_test.shape[0], 500):
    X_test2 = autoencoder_features(theta, hidden_size, visible_size, X_test[i:i+500], step=8, patch_size=8)
    y_test2 = test['y'][i:i+500].ravel()
    sum_tree += (np.sum(trees2.predict(X_test2) == y_test2))
    sum_svm += (np.sum(svm2.predict(X_test2) == y_test2))
print(sum_tree/X_test.shape[0])
print(sum_svm/X_test.shape[0])

0.43275
0.427875


C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


In [13]:
del X_test
del y_test
del X_test2
del y_test2
del test

**Обучение рандомизированных деревьев и алгоритма опорных векторов. Признаки получены с помощью однослойного автокодировщика. Шаг = 4**

In [14]:
f = open('./data/train.pk', 'rb')
train = pickle.load(f)
f.close()

In [15]:
X3 = autoencoder_features(theta, hidden_size, visible_size, train['X'], step=4, patch_size=8)
y3 = train['y'].ravel()
trees3 = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=3)
svm3 = sklearn.svm.LinearSVC()
trees3.fit(X3, y3)
svm3.fit(X3, y3)

C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [16]:
del X3
del y3
del train

In [18]:
f = open('./data/test.pk', 'rb')
test = pickle.load(f)
f.close()

In [19]:
X_test = test['X']
y_test = test['y'].ravel()
sum_tree3 = 0
sum_svm3 = 0
for i in range(0, X_test.shape[0], 500):
    X_test3 = autoencoder_features(theta, hidden_size, visible_size, X_test[i:i+500], step=4, patch_size=8)
    y_test3 = test['y'][i:i+500].ravel()
    sum_tree3 += (np.sum(trees3.predict(X_test3) == y_test3))
    sum_svm3 += (np.sum(svm3.predict(X_test3) == y_test3))
print(sum_tree3/X_test.shape[0])
print(sum_svm3/X_test.shape[0])

0.433
0.475875


C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


In [20]:
del X_test
del y_test
del X_test3
del y_test3
del test

** Обучение параметров трёхслойного автокодировщика**

In [32]:
reload(aec)
hidden_size = np.array([100, 75, 100])
visible_size = 192
lambda_ = 1e-6
sparsity_param = 1e-2
beta = 3
theta = aec.initialize(hidden_size, visible_size)
J = lambda theta: aec.autoencoder_loss(theta, visible_size, hidden_size, lambda_, \
                                       sparsity_param, beta, data[:10000])
optimal = scipy.optimize.minimize(J, theta, method='L-BFGS-B', jac=True, \
                                  options={'disp' : True, 'maxiter' : 2000})
theta = optimal['x']
np.savez('theta_3.npz', theta=theta)

In [33]:
del data
f = open('./data/train.pk', 'rb')
train = pickle.load(f)
f.close()

**Обучение рандомизированных деревьев и алгоритма опорных векторов. Признаки получены с помощью трёхслойного автокодировщика. Шаг = 8**

In [34]:
X4 = autoencoder_features(theta, hidden_size, visible_size, train['X'], step=8, patch_size=8)
y4 = train['y'].ravel()
trees4 = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=3)
svm4 = sklearn.svm.LinearSVC()
trees4.fit(X4, y4)
svm4.fit(X4, y4)

C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [35]:
del X4
del y4
del train
f = open('./data/test.pk', 'rb')
test = pickle.load(f)
f.close()

In [36]:
X_test = test['X']
y_test = test['y'].ravel()
sum_tree4 = 0
sum_svm4 = 0
for i in range(0, X_test.shape[0], 500):
    X_test4 = autoencoder_features(theta, hidden_size, visible_size, X_test[i:i+500], step=8, patch_size=8)
    y_test4 = test['y'][i:i+500].ravel()
    sum_tree4 += (np.sum(trees4.predict(X_test4) == y_test4))
    sum_svm4 += (np.sum(svm4.predict(X_test4) == y_test4))
print(sum_tree4/X_test.shape[0])
print(sum_svm4/X_test.shape[0])

0.399625
0.342125


C:\prak\prak\lab5\autoencoder.py:85: RuntimeWarning: overflow encountered in exp
  x = 1 / (1 + np.exp(-x))


In [ ]:
del X_test
del y_test
del X_test4
del y_test4
del test

In [29]:
W = theta
w = W[:192*100].reshape(192, 100)
w = w.T
reload(dl)
dl.display_layer(w[:64], 'check.png')

## ** Результаты экспериментов **

|Признаки|Random Forest| SVM|
|:---:|:---:|:---:|
|Интенсивности цветовых каналов|43.6%|28%|
|Однослойный автокодировщик с шагом 8|43.2%|42%|
|Однослойный автокодировщик с шагом 4|43.3%|47%|
|Трёхслойный автокодировщик с шагом 8|39%|34%|

## ** Выводы **

По проделанным экспериментам можно заключить, что в случае с алгоритмом опорных векторов признаки, полученные однослойным автокодировщиком, улучшают работу агоритма с 28% точности до 42% - 47%. А в случае с рандомизированными деревьями точность почти не изменяется. Трёхслойный автокодировщик генерирует более плохие признаки, чем однослойный, что несоответствует теоретическим предположениям. Скорее всего, это связано с неудачной настройкой параметров трёхслойного автокодиовщика.